# Create Feature Class with the Sheriff Sales values, Add to Employee Map, Add to Sheriff Map, and send an email to Kathy Skinner telling her of the mismatched values.

 <b> TO DO: </b>
 <ul> 
        <li><b><del>Edit Address DataFrame for merge in case the ParcelID merge doesn't have a match.</b></li>
        <li><b>Use spatial join when the parcels do not match. Test spatial join with python api. If not use geopandas spatial join.</b></li>
        <li><b>Do not use arc.env.workspace Environment Variables.</b></li>
        <li><b>Import dotenv to get rid of login credentials.</b></li>
        
        
        
         </ul> 

In [ ]:
#Modules to import
import arcpy as arc
import pandas as pd
from arcgis import GIS, features
import csv
# import pendulum
from pathlib import Path
from shutil import copy2 as cp
import numpy as np

### FOR USE WITH NOTEBOOK ONLY

In [ ]:
pd.options.display.max_columns = 40

## Curate the CSV file to be used with ArcGIS

In [ ]:
#create Variables
fMon = pendulum.datetime(2021, 4, 1)
fMon_Str = fMon.strftime('%m/%d/%Y')
today = pendulum.today()
tStr = today.strftime('%m/%d/%Y')
# print(tStr)
now = pendulum.now()
mDate = now.strftime('%m%Y')
dDate =  now.strftime('%m_%d')
uPath = Path.home()
revPath = uPath / 'GIS' / 'Review'
gisPath = uPath / 'GIS' / 'Processing'
netDir = Path(r'\\esripub2\SalesWebExport')
netCsv = netDir / 'SalesWebExport.csv'
sPath = revPath / 'Sheriff' / f'{dDate}'
sPath.mkdir(parents=True, exist_ok=True)
dropFields = ['StreetNumber', 'StreetPrefix', 'Address1', 'StreetType']

In [ ]:
#Copy the csv file from network drive to local drive for processing and create DataFrame 
try:
    cp(netCsv, gisPath)
    print(f'Copied {netCsv} to {gisPath}')
except:
    print(f'Could not copy {netCsv}')
# in the script check to make sure that the user has access to the server and can copy the file

In [ ]:
#can use Pandas, CSV module, or generators/iterators
lines = (line for line in netCsv.open())
list_line = (s.rstrip().split(',') for s in lines)
cols = next(list_line)  
# print(list(data))
sher_dict = (dict(zip(cols,data)) for data in list_line)
d = [k for k,v in sher_dict]
print(d) 
# jAmount = (p['"ApproxJudgment"'] for p in sher_dict)
# l = [k for k,v in sher_dict]
# print(f'Blank {l}.')


In [ ]:
print(list(cols))

In [ ]:
sales_df = pd.read_csv(netCsv, na_filter= False)
print(f"Created DF from {netCsv.name}.")

In [ ]:
#Manage Data by changing the Prefix to match the Address Format in the Address Feature Class
columns = sales_df.columns.to_list()
# print(columns)
pre = sales_df.StreetPrefix.unique()
# print(pre)
for p in pre:
    if p.upper() == 'NORTH':
        sales_df['StreetPrefix'] = sales_df.StreetPrefix.apply(lambda x: x.replace(p, 'N').strip())
    elif p.upper() == 'EAST':
        sales_df['StreetPrefix'] = sales_df.StreetPrefix.apply(lambda x: x.replace(p, 'E').strip())
    elif p.upper() == 'WEST':
        sales_df['StreetPrefix'] = sales_df.StreetPrefix.apply(lambda x: x.replace(p, 'W').strip())
    elif p.upper() == 'SOUTH':
        sales_df['StreetPrefix'] = sales_df.StreetPrefix.apply(lambda x: x.replace(p, 'S').strip())
    else:
        sales_df.StreetPrefix
# sales_df.head()

In [ ]:
#Manage Data by changing the Prefix to match the Address Format in the Address Feature Class
sType = sales_df.StreetType.unique()
# print(sType)
for d in sType:
    if d.upper() == 'ROAD':
        sales_df['StreetType'] = sales_df.StreetType.apply(lambda x: x.replace(d, 'RD').strip())
    elif d.upper() == 'DRIVE':
        sales_df['StreetType'] = sales_df.StreetType.apply(lambda x: x.replace(d, 'DR').strip())
    elif d.upper() == 'BOULEVARD':
        sales_df['StreetType'] = sales_df.StreetType.apply(lambda x: x.replace(d, 'BLVD').strip())
    elif d.upper() == 'LANE':
        sales_df['StreetType'] = sales_df.StreetType.apply(lambda x: x.replace(d, 'LN').strip())
    elif d.upper() == 'COURT':
        sales_df['StreetType'] = sales_df.StreetType.apply(lambda x: x.replace(d, 'CT').strip())
    elif d.upper() == 'CIRCLE':
        sales_df['StreetType'] = sales_df.StreetType.apply(lambda x: x.replace(d, 'CIR').strip())
    elif d.upper() == 'STREET':
        sales_df['StreetType'] = sales_df.StreetType.apply(lambda x: x.replace(d, 'ST').strip())
    elif d.upper() == 'HIGHWAY':
        sales_df['StreetType'] = sales_df.StreetType.apply(lambda x: x.replace(d, 'HWY').strip())
    elif d.upper() == 'AVENUE':
        sales_df['StreetType'] = sales_df.StreetType.apply(lambda x: x.replace(d, 'AVE').strip())
    elif d.upper() == 'PLACE':
        sales_df['StreetType'] = sales_df.StreetType.apply(lambda x: x.replace(d, 'PL').strip())
    else:
        sales_df.StreetType.str.strip()
# sales_df.head()

In [ ]:
# Create Address from other fields and remove white space
# for d in pre:
#     if d == ' ':
#         sales_df['Address'] = sales_df.StreetNumber + ' ' + sales_df.Address1 + ' ' + sales_df.StreetType      
#     else:
#         sales_df['Address'] = sales_df.StreetNumber + ' ' + sales_df.StreetPrefix + ' ' + sales_df.Address1 + \
#         ' ' + sales_df.StreetType
# sourceCol = sales_df.columns.get_loc('StreetNumber')
sales_df['Add'] = sales_df[['StreetNumber', 'StreetPrefix', 'Address1', 'StreetType']].values.tolist()
sales_df['Addr'] = sales_df['Add'].apply(' '.join)
sales_df['Address'] = sales_df['Addr'].apply(lambda x: ' '.join(x.split()))
# sales_df.iloc[sales_df[''].apply
sales_df.drop(columns=['Add', 'Addr'], inplace=True)
# sales_df.assign(Address = )
# sales_df.head()

In [ ]:
#If Address2 field is notnull or na then move value into Address field. Do this after address matching when using df_add to merge.
if len(sales_df.loc[sales_df.Address2 != '']) > 0:
    sales_df.Address = np.where(sales_df.Address2 != '', sales_df.Address2, sales_df.Address)

In [ ]:
#Check dates and remove dates that are before current date
sales_df['PropStatus'] = sales_df.PropertyStatusDate
sales_df.PropertyStatusDate = pd.to_datetime(sales_df.PropertyStatusDate, format='%m/%d/%Y', errors='ignore', exact=True)

#Change fMon_Str to tStr when removing by current date
sales_df = sales_df.loc[sales_df.PropertyStatusDate >= fMon_Str, :]
sales_df.drop(columns='PropertyStatusDate', inplace=True)

In [ ]:
sales_df.columns

In [ ]:
#Filter out certain action types and property statuses
sales_df = sales_df.loc[sales_df.ActionTypeDescription != 'Personal Property']
sales_df = sales_df.loc[sales_df.PropertyStatusDescription != 'Stayed']

#need to create a note section if the parcel number contains a &. Call count to see if there are any values that meet this requirement. Need to remove the & from the column
sValue = sales_df.ParcelNumber.str.count('\&').tolist()
for s in sValue:
    if s == 1:
        sales_df['Notes'] = sales_df.loc[sales_df.ParcelNumber.str.contains('\&')].values + ': This parcel is also included in the listed sale'
    else:
        print('There are no additional parcels.')
# sales_df

In [ ]:
#Holds all the values with a Parcel Number
sales_df.ParcelNumber = sales_df.ParcelNumber.replace('', np.nan)
sales_df
#Handle missing Parcel Number values
nan_df = sales_df.loc[sales_df.ParcelNumber.isna(), :]
print(len(nan_df))

In [ ]:
#variable to hold the number of values in each df to not forget any
sValues = sales_df.ParcelNumber.count().tolist()
print(sValues)
nValues = nan_df.SheriffNumber.count().tolist()
print(nValues)

In [ ]:
#Create CSV for nan values


In [ ]:
# sales_df_test['Address'] = sales_df_test['Address'].apply(lambda x: x.strip())
# lol = sales_df.Addr.unique().tolist()
# print(lol)
# for x in lol:
#     print(x + ' ' + str(len(x)))
# for x in lol:
#     a = ' '.join(x.split())
#     print(x + ' ' + str(len(a)))

Check to see if there are any parcel #'s with an & in the field. If so writes the value to a new column called "Notes"

In [ ]:
# x = '9-00-1111 & aksdasd0asdasd'
# print(x.split('&')[-1])
# ePar = sales_df.ParcelNumber.unique()
# for p in ePar:
#     if len(p.split('&')[-1]) > 1:
#         print(p)
#     else:
#         print('Nope')
idx = sales_df.ParcelNumber.apply(lambda x: x.split('&')[-1])
# print(idx)
sales_df['Notes'] = ''
sales_df.loc[idx, 'Notes'] = str(sales_df.ParcelNumber.str.contains('&', na=False)).split('&')[-1] + ': This parcel is also included in the listed sale'
# sales_df
# sales_df

In [ ]:
#Addressing and Parcels using spatial DataFrame from arcgis to merge
#Need to change these from the hard values when running the script
locGDB = gisPath / 'Data_042021.gdb' / f'Daily_{dDate}'
gdbPar = locGDB / f'ParcelFabric_Parcels_{dDate}'
gdbAdd = locGDB / f'Addressing_{dDate}'

df_add = features.GeoAccessor.from_featureclass(gdbAdd)
exp = "TYPE = 7 AND Historical = 0"
df_par = features.GeoAccessor.from_featureclass(gdbPar, where_clause=exp, fields=['Name'])

In [ ]:
#Create DataFrame from merge with parcels and sales
merge_par = sales_df.merge(df_par, how='outer', right_on='Name', left_on='ParcelNumber', indicator=True)
# merge_par.columns
iList = merge_par._merge.unique().tolist()
print(iList)
#Create csvs from merge for review later
for i in iList:
    if i == 'right_only':
        a = sPath / 'Par_Sales_Right.csv'
        merge_par.loc[merge_par._merge == 'right_only', :].to_csv(a, index=False)
        print(f'Created {a.stem}')
    elif i == 'left_only':
        a = sPath / 'Par_Sales_Left.csv'
        merge_par.loc[merge_par._merge == 'left_only', :].to_csv(a, index=False)
        print(f'Created {a.stem}')
    elif i == 'both':
        a = sPath / 'Par_Sales_Both.csv'
        merge_par.loc[merge_par._merge == 'both', :].to_csv(a, index=False)
        print(f'Created {a.stem}')
# rCSV = sPath / 'Par_Sales_Right.csv'
# lCSV = sPath / 'Par_Sales_Left.csv'
# bCsv = sPath / 'Par_Sales_Both.csv'
# merge_par.loc[merge_par._merge == 'right_only', :].to_csv(index=False)

#Number of values from merge 
pValues = len(merge_par.loc[merge_par._merge == 'both', :])
print(pValues)

if pValues == sValues:
    merge_par_b = merge_par.loc[merge_par._merge == 'both', :]
    merge_par_b.drop(columns=['_merge', 'StreetNumber', 'StreetPrefix', 'StreetType', 'StreetSuffix', 'Address1', 'Address2', 'Address3', 'Name'], inplace=True)
    print('All Features are matching')

In [ ]:
#Trying to get the fields to match. Most likely will create new feature classes to use. 
netDir = Path('G:\\MasterGISFiles\\Ben')
netDB = netDir / 'GISPro' / 'SDE Connections'
iE = netDB / 'MAPPINGADMIN.sde' / 'PROD.MAPPINGADMIN.ParcelEditing'
arc.env.workspace = f'{iE.parent}'
# print(locGDB)
fcList = [f for f in arc.ListFeatureClasses(feature_dataset='PROD.GISADMIN.SheriffSales')]
print(fcList)
fList = [f.name for f in arc.ListFields(f'{iE.parent / "PROD.GISADMIN.SheriffSales" / "PROD.GISADMIN.BothSales"}')]
# for x in fcList:
#     a = arc.ListFields(f'{iE.parent / 'PROD.GISADMIN.SheriffSales' / x}')
#     for l in a:
#         fList.
#         fList.append(arc.ListFields(x))
print(fList)

sCols = sales_df.columns.tolist()
cols = dict(zip(sCols, fList))
print(cols)

In [ ]:
merge_par_b.columns

In [ ]:
# merge_par.drop(columns=['StreetNumber', 'StreetPrefix', 'StreetType', 'StreetSuffix', 'Address1', 'Address2', 'Address3', 'Name'], inplace=True)
# merge_par = merge_par[['SheriffNumber'] + [col for col in merge_par.columns if col !='SheriffNumber']]
# print(list(merge_par_b))
# cols = list(merge_par_b)

If values from merge is zero then use df_add and create spatial join <br>
If values from sales_df = values from df_par then create Sheriff feature class

In [ ]:
shFC = locGDB / f'Sheriff_Final_{dDate}'
if pValues == sValues:
    merge_par_b.spatial.to_featureclass(shFC, sanitize_columns=False)
    print(f"Created {shFC.stem} at {locGDB}")

### Edit the Address DataFrame (df_add) to replace the null values in FullAddr column. Only use if the Parcel Fabric Layer and the sales speadsheet don't fully match.

In [ ]:
# df_add['STS'].replace('None', " ")
# df_add.loc[df_add.STS == None, :]
# df_add.isnull().sum()
df_add['STS'].fillna(np.nan, inplace= True)
df_add['STS'].replace(' ', '', inplace=True)
df_add['STS'].replace(np.nan, '', inplace=True)
# df_add['STS'].unique()
# df_add.head()

In [ ]:
aType = df_add.STS.unique()
# print(sType)
for d in aType:
    if d.upper() == 'ROAD':
        df_add.STS = df_add.STS.apply(lambda x: x.replace(d, 'RD').strip())
    elif d.upper() == 'DRIVE':
        df_add.STS = df_add.STS.apply(lambda x: x.replace(d, 'DR').strip())
        # sales_df['StreetType'] = sales_df.StreetType.apply(lambda x: x.replace(d, 'DR').strip())
    elif d.upper() == 'BOULEVARD':
        df_add.STS = df_add.STS.apply(lambda x: x.replace(d, 'BLVD').strip())
        # sales_df['StreetType'] = sales_df.StreetType.apply(lambda x: x.replace(d, 'BLVD').strip())
    elif d.upper() == 'LANE':
        df_add.STS = df_add.STS.apply(lambda x: x.replace(d, 'LN').strip())
        # sales_df['StreetType'] = sales_df.StreetType.apply(lambda x: x.replace(d, 'LN').strip())
    elif d.upper() == 'COURT':
        df_add.STS = df_add.STS.apply(lambda x: x.replace(d, 'CT').strip())
        # sales_df['StreetType'] = sales_df.StreetType.apply(lambda x: x.replace(d, 'CT').strip())
    elif d.upper() == 'CIRCLE':
        df_add.STS = df_add.STS.apply(lambda x: x.replace(d, 'CIR').strip())
        # sales_df['StreetType'] = sales_df.StreetType.apply(lambda x: x.replace(d, 'CIR').strip())
    elif d.upper() == 'STREET':
        df_add.STS = df_add.STS.apply(lambda x: x.replace(d, 'ST').strip())
        # sales_df['StreetType'] = sales_df.StreetType.apply(lambda x: x.replace(d, 'ST').strip())
    elif d.upper() == 'HIGHWAY':
        df_add.STS = df_add.STS.apply(lambda x: x.replace(d, 'HWY').strip())
        # sales_df['StreetType'] = sales_df.StreetType.apply(lambda x: x.replace(d, 'HWY').strip())
    elif d.upper() == 'AVENUE':
        df_add.STS = df_add.STS.apply(lambda x: x.replace(d, 'AVE').strip())
        # sales_df['StreetType'] = sales_df.StreetType.apply(lambda x: x.replace(d, 'AVE').strip())
    elif d.upper() == 'PLACE':
        df_add.STS = df_add.STS.apply(lambda x: x.replace(d, 'PL').strip())
        # sales_df['StreetType'] = sales_df.StreetType.apply(lambda x: x.replace(d, 'PL').strip())
    else:
        df_add.STS.str.strip()
print(df_add.STS.unique())

In [ ]:
#Spatially Join Addressing and Parcels. Need to project Addressing to the same SR
# df_add = df_add.spatial.project()
# df_join = df_par.spatial.join(df_add, left_tag='p_', right_tag='a_')
# df_join

In [ ]:
df_add.fillna(np.nan, inplace= True)
# df_add.replace(' ', '', inplace=True)
df_add.replace(np.nan, '', inplace=True)
# print(df_add.columns)
print(df_add.STS.value_counts())

In [ ]:
df_add.SAN = df_add.SAN.astype('str',errors='ignore')
df_add.SAN = df_add.SAN.apply(lambda x: x.split('.')[0])
df_add['FAddress'] = df_add[['SAN', 'PRD', 'STN', 'STS', 'POD']].values.tolist()
# df_add.FAddress.values
df_add['FooAddress'] = df_add['FAddress'].apply(' '.join)
df_add['FoAddress'] = df_add['FooAddress'].apply(lambda x: ' '.join(x.split()))
# df_add

In [ ]:
df_add.FAddress = df_add.FoAddress
kColumns = ['FAddress', 'SHAPE']
# print([x for x in kColumns if x not in kColumns])
df_add.drop(columns=[x for x in df_add if x not in kColumns], inplace=True)
df_add

Create DataFrame from merge with address and sales, create csvs from merge for review later, and get number of values after the merge.

In [ ]:
#Create DataFrame from merge with address and sales
sales_df['lAdd'] = sales_df.Address.str.upper()
# del merge_add
merge_add = df_add.merge(sales_df, how='outer', left_on='FAddress', right_on='lAdd', indicator=True, )
# merge_add

#create csvs from merge for review later

merge_add[merge_add._merge == 'both']

#Number of Values from merge
maValues = merge_add.SHAPE.count().tolist()

Use gdbJoin after editing and joining the address and sales tables.

In [ ]:
gdbJoin = locGDB / f'Par_Add_{dDate}'
arc.SpatialJoin_analysis(f'{gdbPar}', f'{gdbAdd}', f'{gdbJoin}')
df_join = features.GeoAccessor.from_featureclass(gdbJoin)

In [ ]:
#Remove Columns from df_par not needed for merge
# pkColumns = ['Name', 'SHAPE']
# df_par.drop(columns=[x for x in df_par if x not in pkColumns], inplace=True)
# df_par.columns